## Installation

In [ ]:
!pip install kor

In [4]:
# Kor!
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number

# LangChain Models
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# Standard Helpers
import json
import os

# For token counting
from langchain.callbacks import get_openai_callback


def printOutput(output):
    print(json.dumps(output,sort_keys=True, indent=3))

In [6]:
os.environ["OPENAI_API_KEY"] = input("OpenAI Key: ")

### LLM Model & Schema Function

In [28]:
# define model
llm = ChatOpenAI(
    model_name="gpt-4", # "gpt-3.5-turbo", "gpt-4"
    temperature=0,
)

In [46]:
def DefineSchema(id_name:str, description:tuple, atr_list:list):
  """
  id = any key value as string
  description = Give a solid explanation to the model what you looking for
  attributes = Given list that contain id, description and few example (more is better) in order.
  """
  id=id_name
  description=description

  attributes=[]
  for atr in atr_list:
    try:
      id = atr[0]
    except:
      print("'key' is expected in string format as a first argument of your list")

    try:
      description = atr[1]
    except:
      print("'description' of your key is expected in string format as a second argument of your list ")

    try:
      examples = atr[2]
    except:
      examples=None
      print(f"'examples' is not given for key: '{id}'. For better performance give an few sample in tuple format as a third argument of your list")


    # define text
    if examples != None:
      attributes.append(Text(id=id,
          description=description,
          examples=examples,
          many=True,
          ))
    else:
      attributes.append(Text(id=id,
          description=description,
          many=True,
          ))
    return schema

### Run an Example

In [51]:
# Define search prompt
atr_list = [
 ["date",
  "year the user wants",
   [("please increase 2014's customers by 15%", "2014"),
    ("raise %2 of the price at the at of the 2020", "2020")]],
 ["metric",
  "The unit or metric a user would like to influence",
   [("please increase 2014's customers by 15%", "customers")]],
 ["amount",
  "The quantity of a forecast adjustment",
   [("please increase 2014's customers by 15%", ".15")]]
 ]


schema = DefineSchema(id_name="forecaster",
                      description=(
                          "User is controling an app that makes financial forecasts. "
                          "They will give a command to update a forecast in the future"
                      ),
                      atr_list=atr_list)

In [52]:
# Create LLM chain
chain = create_extraction_chain(llm, schema, encoder_or_encoder_class='json')

In [53]:
t1 = "decrease 7.12% discount to 2021"

t2 = "please add 1.75 dolar to the sale price at 2017"

t3 = "please add 15 more units sold to 2023"

for text in [t1,t2,t3]:
  with get_openai_callback() as cb:
    output = chain.run(text=(text))["data"]
    print(f"Total Cost (USD): ${cb.total_cost}")
  printOutput(output)
  print("\n\n")

Total Cost (USD): $0.01182
{
   "forecaster": {
      "amount": [
         "7.12%"
      ],
      "metric": [
         "discount"
      ],
      "year": [
         "2021"
      ]
   }
}



Total Cost (USD): $0.011940000000000001
{
   "forecaster": {
      "amount": [
         "1.75"
      ],
      "metric": [
         "sale price"
      ],
      "year": [
         "2017"
      ]
   }
}



Total Cost (USD): $0.01167
{
   "forecaster": {
      "amount": [
         "15"
      ],
      "metric": [
         "units sold"
      ],
      "year": [
         "2023"
      ]
   }
}



